# SEC Financial Data Project 

The data used for this project comes from the __[SEC's Office of Structured Data](https://www.sec.gov/dera/data/financial-statement-data-sets)__. I downloaded their quarterly filings from Q1 2022 to Q1 2023. Initially I had only imported the 2022 quarters but I noticed that there seemed to be a lag where some companies' Q4 2022 data was only showing up in the Q1 2023 data, so I brought that in to get a full population.

In [1]:
# import pandas and read csv files as dataframes
import pandas as pd
num2022Q4 = pd.read_csv('Datasource/num2022Q4.txt',sep='\t', low_memory=False)
num2022Q3 = pd.read_csv('Datasource/num2022Q3.txt',sep='\t', low_memory=False)
num2022Q2 = pd.read_csv('Datasource/num2022Q2.txt',sep='\t', low_memory=False)
num2022Q1 = pd.read_csv('Datasource/num2022Q1.txt',sep='\t', low_memory=False)
num2023Q1 = pd.read_csv('Datasource/num2023Q1.txt',sep='\t', low_memory=False)
num2022Q4.head()

,adsh,tag,version,coreg,ddate,qtrs,uom,value,footnote
0,0001140361-22-040133,AcceleratedShareRepurchaseProgramAdjustment,us-gaap/2021,NaN,20220331,1,USD,-150000000.0,NaN
1,0001140361-22-040134,AcceleratedShareRepurchaseProgramAdjustment,us-gaap/2021,NaN,20220331,1,USD,-150000000.0,NaN
2,0001140361-22-040134,AcceleratedShareRepurchaseProgramAdjustment,us-gaap/2021,NaN,20220630,1,USD,0.0,NaN
3,0001628280-22-028255,AcceleratedShareRepurchaseProgramAdjustment,us-gaap/2021,NaN,20220930,1,USD,20010000.0,NaN
4,0000860413-22-000202,AcceleratedShareRepurchasesAdjustmentToRecorde...,us-gaap/2021,NaN,20220930,3,USD,197400000.0,NaN


In [2]:
# stack dataframes on top of one another for one dataframe with 5 time periods
num_data = pd.concat([num2022Q4, num2022Q3, num2022Q2, num2022Q1, num2023Q1])


In [3]:
num_data.dtypes


adsh         object
tag          object
version      object
coreg        object
ddate         int64
qtrs          int64
uom          object
value       float64
footnote     object
dtype: object

In [4]:
num_data.head()


,adsh,tag,version,coreg,ddate,qtrs,uom,value,footnote
0,0001140361-22-040133,AcceleratedShareRepurchaseProgramAdjustment,us-gaap/2021,NaN,20220331,1,USD,-150000000.0,NaN
1,0001140361-22-040134,AcceleratedShareRepurchaseProgramAdjustment,us-gaap/2021,NaN,20220331,1,USD,-150000000.0,NaN
2,0001140361-22-040134,AcceleratedShareRepurchaseProgramAdjustment,us-gaap/2021,NaN,20220630,1,USD,0.0,NaN
3,0001628280-22-028255,AcceleratedShareRepurchaseProgramAdjustment,us-gaap/2021,NaN,20220930,1,USD,20010000.0,NaN
4,0000860413-22-000202,AcceleratedShareRepurchasesAdjustmentToRecorde...,us-gaap/2021,NaN,20220930,3,USD,197400000.0,NaN


In [5]:
# rename and title case column headers
num_data = num_data.rename(columns=
                {"adsh":"Accession Number","tag":"Account","ddate":"End Date","uom":"Unit of Measure","qtrs":"Quarters Included","coreg":"Co-Registrant"})
num_data.columns = num_data.columns.str.title()
num_data.head()

,Accession Number,Account,Version,Co-Registrant,End Date,Quarters Included,Unit Of Measure,Value,Footnote
0,0001140361-22-040133,AcceleratedShareRepurchaseProgramAdjustment,us-gaap/2021,NaN,20220331,1,USD,-150000000.0,NaN
1,0001140361-22-040134,AcceleratedShareRepurchaseProgramAdjustment,us-gaap/2021,NaN,20220331,1,USD,-150000000.0,NaN
2,0001140361-22-040134,AcceleratedShareRepurchaseProgramAdjustment,us-gaap/2021,NaN,20220630,1,USD,0.0,NaN
3,0001628280-22-028255,AcceleratedShareRepurchaseProgramAdjustment,us-gaap/2021,NaN,20220930,1,USD,20010000.0,NaN
4,0000860413-22-000202,AcceleratedShareRepurchasesAdjustmentToRecorde...,us-gaap/2021,NaN,20220930,3,USD,197400000.0,NaN


In [6]:
# import numpy - np.select to assign Quarter values to dates
import numpy as np
conditions = [(num_data['End Date'] == 20211231),(num_data['End Date'] == 20220331),(num_data['End Date'] == 20220630),(num_data['End Date'] == 20220930),(num_data['End Date'] == 20221231)]
values = ['Q4_2021', 'Q1_2022', 'Q2_2022', 'Q3_2022', 'Q4_2022']
num_data['Quarter'] = np.select(conditions, values)

In [7]:
num_data = num_data.drop(['Co-Registrant', 'Footnote'], axis=1)


## Completeness check
<p>The pivot in the cell below was necessary in figuring out the completeness of my population. This was where I originally discovered there was a lag in the data because Q4 2022 had with a significantly smaller balance than the other quarters, so I knew data was missing.</p>

In [8]:
#pivot dataframe to make sure each quarter was correctly assigned in step 6
num_by_quarter = pd.pivot_table(num_data,
    index = None,
    columns = 'Quarter',
    values = 'Value',
    aggfunc = 'mean'
    )
num_by_quarter


Quarter,0,Q1_2022,Q2_2022,Q3_2022,Q4_2021,Q4_2022
Value,3.917549e+10,4.111100e+10,3.163668e+10,8.925029e+11,1.747312e+11,1.163174e+10


In [9]:
num_data.dtypes


Accession Number      object
Account               object
Version               object
End Date               int64
Quarters Included      int64
Unit Of Measure       object
Value                float64
Quarter               object
dtype: object

In [10]:
# remove rows that did not get a quarter assigned in step 6 (date is not quarter end)
num_data = num_data[num_data['Quarter'] != '0']


## Import filer information
<p>These "sub" files in the next cell contain information on every entity that files with the SEC. They are each assigned an Accession Number or "adsh" which was also a field in the previous "num" files which I could use as a unique identifier to join the two datasets. </p>

In [11]:
# read CSVs of company data
sub2022Q1 = pd.read_csv('Datasource/sub2022Q1.txt',sep='\t')
sub2022Q4 = pd.read_csv('Datasource/sub2022Q4.txt',sep='\t')
sub2022Q3 = pd.read_csv('Datasource/sub2022Q3.txt',sep='\t')
sub2022Q2 = pd.read_csv('Datasource/sub2022Q2.txt',sep='\t')
sub2023Q1 = pd.read_csv('Datasource/sub2023Q1.txt',sep='\t')


In [12]:
sub_data = pd.concat([sub2022Q4, sub2022Q3, sub2022Q2, sub2022Q1, sub2023Q1])
sub_data.head()


,adsh,cik,name,sic,countryba,stprba,cityba,zipba,bas1,bas2,...,period,fy,fp,filed,accepted,prevrpt,detail,instance,nciks,aciks
0,0000014707-22-000072,14707,CALERES INC,3140.0,US,MO,ST LOUIS,63105,8300 MARYLAND AVE,P O BOX 29,...,20221031.0,2022.0,Q3,20221206,2022-12-06 14:50:00.0,0,1,cal-20221029x10q_htm.xml,1,NaN
1,0000014846-22-000059,14846,BRT APARTMENTS CORP.,6798.0,US,NY,GREAT NECK,11021-3190,60 CUTTER MILL RD,SUITE 303,...,20221130.0,NaN,NaN,20221206,2022-12-06 16:16:00.0,0,0,brt-20221206_htm.xml,1,NaN
2,0000018349-22-000191,18349,SYNOVUS FINANCIAL CORP,6021.0,US,GA,COLUMBUS,31901,1111 BAY AVENUE,STE 500,...,20221130.0,NaN,NaN,20221206,2022-12-06 16:16:00.0,0,0,syn-20221206_htm.xml,1,NaN
3,0000035527-22-000248,35527,FIFTH THIRD BANCORP,6022.0,US,OH,CINCINNATI,45263,38 FOUNTAIN SQ PLZ,FIFTH THIRD CENTER,...,20221130.0,NaN,NaN,20221206,2022-12-06 17:01:00.0,0,0,fitb-20221206_htm.xml,1,NaN
4,0000041719-22-000063,41719,GLATFELTER CORP,2621.0,US,NC,CHARLOTTE,28209,4350 CONGRESS STREET,SUITE 600,...,20221130.0,NaN,NaN,20221206,2022-12-06 16:47:00.0,0,0,glt-20221202_htm.xml,1,NaN


In [16]:
# simplify company data to just columns needed
filer_info = sub_data[['adsh','cik','name','sic','period','fy','fp','fye']]
filer_info.head()


,adsh,cik,name,sic,period,fy,fp,fye
0,0000014707-22-000072,14707,CALERES INC,3140.0,20221031.0,2022.0,Q3,131.0
1,0000014846-22-000059,14846,BRT APARTMENTS CORP.,6798.0,20221130.0,NaN,NaN,1231.0
2,0000018349-22-000191,18349,SYNOVUS FINANCIAL CORP,6021.0,20221130.0,NaN,NaN,1231.0
3,0000035527-22-000248,35527,FIFTH THIRD BANCORP,6022.0,20221130.0,NaN,NaN,1231.0
4,0000041719-22-000063,41719,GLATFELTER CORP,2621.0,20221130.0,NaN,NaN,1231.0


In [17]:
# join account data with company data so we can see what company all balances relate to
# joined on 'Accession Number' which is unique to each filing
SEC_data = num_data.set_index('Accession Number').join(filer_info.set_index('adsh'))
SEC_data.head()


,Account,Version,End Date,Quarters Included,Unit Of Measure,Value,Quarter,cik,name,sic,period,fy,fp,fye
Accession Number,,,,,,,,,,,,,,
0000002178-22-000033,AccountsPayableCurrent,us-gaap/2021,20211231,0,USD,168224000.0,Q4_2021,2178,"ADAMS RESOURCES & ENERGY, INC.",5172.0,20211231.0,2021.0,FY,1231.0
0000002178-22-000033,AccountsReceivableNetCurrent,us-gaap/2021,20211231,0,USD,137789000.0,Q4_2021,2178,"ADAMS RESOURCES & ENERGY, INC.",5172.0,20211231.0,2021.0,FY,1231.0
0000002178-22-000033,AccountsReceivableRelatedPartiesCurrent,us-gaap/2021,20211231,0,USD,2000.0,Q4_2021,2178,"ADAMS RESOURCES & ENERGY, INC.",5172.0,20211231.0,2021.0,FY,1231.0
0000002178-22-000033,AccruedInsuranceCurrentAndNoncurrent,us-gaap/2021,20211231,0,USD,50000.0,Q4_2021,2178,"ADAMS RESOURCES & ENERGY, INC.",5172.0,20211231.0,2021.0,FY,1231.0
0000002178-22-000033,AdditionalPaidInCapital,us-gaap/2021,20211231,0,USD,16913000.0,Q4_2021,2178,"ADAMS RESOURCES & ENERGY, INC.",5172.0,20211231.0,2021.0,FY,1231.0


In [18]:
# pivot to make sure nothing was lost in join - compare to pivot in #8
SEC_data_by_quarter = pd.pivot_table(SEC_data,
    index = None,
    columns = 'Quarter',
    values = 'Value',
    aggfunc = 'mean'
    )
SEC_data_by_quarter


Quarter,Q1_2022,Q2_2022,Q3_2022,Q4_2021,Q4_2022
Value,4.111100e+10,3.163668e+10,8.925029e+11,1.747312e+11,1.163174e+10


In [19]:
SEC_data['sic'].fillna(0, inplace = True)


In [20]:
SEC_data.dtypes


Account               object
Version               object
End Date               int64
Quarters Included      int64
Unit Of Measure       object
Value                float64
Quarter               object
cik                    int64
name                  object
sic                  float64
period               float64
fy                   float64
fp                    object
fye                  float64
dtype: object

In [21]:
SEC_data['sic'] = SEC_data['sic'].astype('int')


## Industry Tagging
<p>The SEC also provides a chart as seen below which maps the "SIC Code" from our filer information files to an Industry. I wanted to join this data in to see if there were any patterns with certain accounts through different industries.</p>

In [22]:
# read CSV with industry codes/titles
SIC_codes = pd.read_csv('Datasource/SIC2.csv')
SIC_codes.head()


,SIC Code,Office,Industry Title
0,100,Industrial Applications and Services,AGRICULTURAL PRODUCTION-CROPS
1,200,Industrial Applications and Services,AGRICULTURAL PROD-LIVESTOCK & ANIMAL SPECIALTIES
2,700,Industrial Applications and Services,AGRICULTURAL SERVICES
3,800,Industrial Applications and Services,FORESTRY
4,900,Industrial Applications and Services,"FISHING, HUNTING AND TRAPPING"


In [23]:
SIC_codes.dtypes


SIC Code           int64
Office            object
Industry Title    object
dtype: object

In [24]:
# join in industry data to see what industry each balance relates to
SEC_full_data = SEC_data.join(SIC_codes.set_index('SIC Code'), on = 'sic')
SEC_full_data.head()


,Account,Version,End Date,Quarters Included,Unit Of Measure,Value,Quarter,cik,name,sic,period,fy,fp,fye,Office,Industry Title
Accession Number,,,,,,,,,,,,,,,,
0000002178-22-000033,AccountsPayableCurrent,us-gaap/2021,20211231,0,USD,168224000.0,Q4_2021,2178,"ADAMS RESOURCES & ENERGY, INC.",5172,20211231.0,2021.0,FY,1231.0,Office of Trade & Services,WHOLESALE-PETROLEUM & PETROLEUM PRODUCTS (NO B...
0000002178-22-000033,AccountsReceivableNetCurrent,us-gaap/2021,20211231,0,USD,137789000.0,Q4_2021,2178,"ADAMS RESOURCES & ENERGY, INC.",5172,20211231.0,2021.0,FY,1231.0,Office of Trade & Services,WHOLESALE-PETROLEUM & PETROLEUM PRODUCTS (NO B...
0000002178-22-000033,AccountsReceivableRelatedPartiesCurrent,us-gaap/2021,20211231,0,USD,2000.0,Q4_2021,2178,"ADAMS RESOURCES & ENERGY, INC.",5172,20211231.0,2021.0,FY,1231.0,Office of Trade & Services,WHOLESALE-PETROLEUM & PETROLEUM PRODUCTS (NO B...
0000002178-22-000033,AccruedInsuranceCurrentAndNoncurrent,us-gaap/2021,20211231,0,USD,50000.0,Q4_2021,2178,"ADAMS RESOURCES & ENERGY, INC.",5172,20211231.0,2021.0,FY,1231.0,Office of Trade & Services,WHOLESALE-PETROLEUM & PETROLEUM PRODUCTS (NO B...
0000002178-22-000033,AdditionalPaidInCapital,us-gaap/2021,20211231,0,USD,16913000.0,Q4_2021,2178,"ADAMS RESOURCES & ENERGY, INC.",5172,20211231.0,2021.0,FY,1231.0,Office of Trade & Services,WHOLESALE-PETROLEUM & PETROLEUM PRODUCTS (NO B...


In [25]:
# pivot to make sure no data was lost in join
SEC_data_by_quarter = pd.pivot_table(SEC_full_data,
    index = None,
    columns = 'Quarter',
    values = 'Value',
    aggfunc = 'mean'
    )
SEC_data_by_quarter


Quarter,Q1_2022,Q2_2022,Q3_2022,Q4_2021,Q4_2022
Value,4.111100e+10,3.163668e+10,8.925029e+11,1.747312e+11,1.163174e+10


In [26]:
# filter for 'Version' = US-GAAP 2021 or 2022 as this is the most relevant financial data to compare accross companies
SEC_full_data = SEC_full_data[(SEC_full_data['Version'].str.contains('us-gaap/2022|us-gaap/2021'))]
SEC_full_data.head()

,Account,Version,End Date,Quarters Included,Unit Of Measure,Value,Quarter,cik,name,sic,period,fy,fp,fye,Office,Industry Title
Accession Number,,,,,,,,,,,,,,,,
0000002178-22-000033,AccountsPayableCurrent,us-gaap/2021,20211231,0,USD,168224000.0,Q4_2021,2178,"ADAMS RESOURCES & ENERGY, INC.",5172,20211231.0,2021.0,FY,1231.0,Office of Trade & Services,WHOLESALE-PETROLEUM & PETROLEUM PRODUCTS (NO B...
0000002178-22-000033,AccountsReceivableNetCurrent,us-gaap/2021,20211231,0,USD,137789000.0,Q4_2021,2178,"ADAMS RESOURCES & ENERGY, INC.",5172,20211231.0,2021.0,FY,1231.0,Office of Trade & Services,WHOLESALE-PETROLEUM & PETROLEUM PRODUCTS (NO B...
0000002178-22-000033,AccountsReceivableRelatedPartiesCurrent,us-gaap/2021,20211231,0,USD,2000.0,Q4_2021,2178,"ADAMS RESOURCES & ENERGY, INC.",5172,20211231.0,2021.0,FY,1231.0,Office of Trade & Services,WHOLESALE-PETROLEUM & PETROLEUM PRODUCTS (NO B...
0000002178-22-000033,AccruedInsuranceCurrentAndNoncurrent,us-gaap/2021,20211231,0,USD,50000.0,Q4_2021,2178,"ADAMS RESOURCES & ENERGY, INC.",5172,20211231.0,2021.0,FY,1231.0,Office of Trade & Services,WHOLESALE-PETROLEUM & PETROLEUM PRODUCTS (NO B...
0000002178-22-000033,AdditionalPaidInCapital,us-gaap/2021,20211231,0,USD,16913000.0,Q4_2021,2178,"ADAMS RESOURCES & ENERGY, INC.",5172,20211231.0,2021.0,FY,1231.0,Office of Trade & Services,WHOLESALE-PETROLEUM & PETROLEUM PRODUCTS (NO B...


In [27]:
# pivot to see how accounts have changed on average over time
account_by_quarter = pd.pivot_table(SEC_full_data,
    index = 'Account',
    columns = 'Quarter',
    values = 'Value',
    aggfunc = 'mean'
    )
account_by_quarter


Quarter,Q1_2022,Q2_2022,Q3_2022,Q4_2021,Q4_2022
Account,,,,,
AcceleratedShareRepurchaseProgramAdjustment,-5.662371e+07,5.400000e+08,2.001000e+07,-1.615450e+07,-6.022385e+07
AcceleratedShareRepurchasesAdjustmentToRecordedAmount,3.010000e+07,4.142667e+07,9.867500e+07,1.686975e+07,7.874700e+07
AcceleratedShareRepurchasesFinalPricePaidPerShare,1.173836e+04,1.969412e+01,3.273600e+01,3.527782e+01,2.960538e+01
AcceleratedShareRepurchasesInitialPricePaidPerShare,2.191250e+01,6.631000e+00,5.488750e+00,1.077000e+01,8.816667e+00
AcceleratedShareRepurchasesSettlementPaymentOrReceipt,1.214286e+08,1.610000e+09,2.000000e+08,1.443750e+08,4.000000e+08
...,...,...,...,...,...
WorkersCompensationDiscountPriorYearAmount,NaN,NaN,NaN,1.387000e+09,NaN
WorkersCompensationLiabilityCurrent,1.641348e+07,1.684925e+07,1.790788e+07,1.889743e+07,1.771222e+07
WorkersCompensationLiabilityCurrentAndNoncurrent,9.509693e+07,8.189475e+07,6.991066e+07,8.564436e+07,6.299630e+07


## Remove duplicate filings
<p>I noticed that my output contained duplicate lines because while some companies' filings werent included until a quarter later, some companies' filings were included in multiple CSVs, so they were being duplicated. In the below cell I use drop_duplicates so that we only have unique filings in our final data set.</p>

In [28]:
# since we needed to bring in 5 CSVs from the SEC to get our full data, it also incuded duplicate accounts - drop these to simplify
SEC_full_data.drop_duplicates()

,Account,Version,End Date,Quarters Included,Unit Of Measure,Value,Quarter,cik,name,sic,period,fy,fp,fye,Office,Industry Title
Accession Number,,,,,,,,,,,,,,,,
0000002178-22-000033,AccountsPayableCurrent,us-gaap/2021,20211231,0,USD,168224000.0,Q4_2021,2178,"ADAMS RESOURCES & ENERGY, INC.",5172,20211231.0,2021.0,FY,1231.0,Office of Trade & Services,WHOLESALE-PETROLEUM & PETROLEUM PRODUCTS (NO B...
0000002178-22-000033,AccountsReceivableNetCurrent,us-gaap/2021,20211231,0,USD,137789000.0,Q4_2021,2178,"ADAMS RESOURCES & ENERGY, INC.",5172,20211231.0,2021.0,FY,1231.0,Office of Trade & Services,WHOLESALE-PETROLEUM & PETROLEUM PRODUCTS (NO B...
0000002178-22-000033,AccountsReceivableRelatedPartiesCurrent,us-gaap/2021,20211231,0,USD,2000.0,Q4_2021,2178,"ADAMS RESOURCES & ENERGY, INC.",5172,20211231.0,2021.0,FY,1231.0,Office of Trade & Services,WHOLESALE-PETROLEUM & PETROLEUM PRODUCTS (NO B...
0000002178-22-000033,AccruedInsuranceCurrentAndNoncurrent,us-gaap/2021,20211231,0,USD,50000.0,Q4_2021,2178,"ADAMS RESOURCES & ENERGY, INC.",5172,20211231.0,2021.0,FY,1231.0,Office of Trade & Services,WHOLESALE-PETROLEUM & PETROLEUM PRODUCTS (NO B...
0000002178-22-000033,AdditionalPaidInCapital,us-gaap/2021,20211231,0,USD,16913000.0,Q4_2021,2178,"ADAMS RESOURCES & ENERGY, INC.",5172,20211231.0,2021.0,FY,1231.0,Office of Trade & Services,WHOLESALE-PETROLEUM & PETROLEUM PRODUCTS (NO B...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0001948056-23-000004,Liabilities,us-gaap/2022,20221231,0,USD,6700163.0,Q4_2022,1948056,KKR INFRASTRUCTURE CONGLOMERATE LLC,6799,20221231.0,2022.0,FY,1231.0,Office of Real Estate & Construction,"INVESTORS, NEC"
0001948056-23-000004,NetAssetValuePerShare,us-gaap/2022,20221231,0,USD,25.0,Q4_2022,1948056,KKR INFRASTRUCTURE CONGLOMERATE LLC,6799,20221231.0,2022.0,FY,1231.0,Office of Real Estate & Construction,"INVESTORS, NEC"
0001948056-23-000004,NetInvestmentIncome,us-gaap/2022,20221231,1,USD,0.0,Q4_2022,1948056,KKR INFRASTRUCTURE CONGLOMERATE LLC,6799,20221231.0,2022.0,FY,1231.0,Office of Real Estate & Construction,"INVESTORS, NEC"


In [29]:
SEC_full_data.to_csv(r"/Users/erintoomey/PycharmProjects/pythonProject/Datasource/SEC_full_data2.csv", index=False)


## Import SOFR rates
After exporting my joined and cleaned data to a CSV in the previous cell, I decided I wanted some market data to identify any patterns over time as interest rates changed. The Federal Reserve has historical SOFR rate data available on their [FRED site](https://fred.stlouisfed.org/series/SOFR).

In [30]:
# read SOFR rate CSV from Federal Reserve
SOFR_rates = pd.read_csv('Datasource/SOFR_rates.csv')


In [31]:
SOFR_rates.head()


,Effective Date,Rate Type,Rate (%),1st Percentile (%),25th Percentile (%),75th Percentile (%),99th Percentile (%),Volume ($Billions),Target Rate From (%),Target Rate To (%),Intra Day - Low (%),Intra Day - High (%),Standard Deviation (%),30-Day Average SOFR,90-Day Average SOFR,180-Day Average SOFR,SOFR Index,Revision Indicator (Y/N),Footnote ID
0,12/30/2022,SOFR,4.3,4.20,4.26,4.33,4.48,1004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12/29/2022,SOFR,4.3,4.20,4.26,4.32,4.45,1037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12/28/2022,SOFR,4.3,4.21,4.27,4.32,4.45,1005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12/27/2022,SOFR,4.3,4.15,4.27,4.32,4.40,1048,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,12/23/2022,SOFR,4.3,4.21,4.26,4.31,4.38,991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
SOFR_rates['Effective Date'] = SOFR_rates['Effective Date'].astype('str')


In [33]:
# use np.select to assign Quarter values to dates
conditions = [(SOFR_rates['Effective Date'] == '12/31/2021'),(SOFR_rates['Effective Date'] == '03/31/2022'),(SOFR_rates['Effective Date'] == '06/30/2022'),(SOFR_rates['Effective Date'] == '09/30/2022'),(SOFR_rates['Effective Date'] == '12/30/2022')]
values = ['Q4_2021', 'Q1_2022', 'Q2_2022', 'Q3_2022', 'Q4_2022']
SOFR_rates['Quarter'] = np.select(conditions, values)

In [34]:
SOFR_rates.head()


,Effective Date,Rate Type,Rate (%),1st Percentile (%),25th Percentile (%),75th Percentile (%),99th Percentile (%),Volume ($Billions),Target Rate From (%),Target Rate To (%),Intra Day - Low (%),Intra Day - High (%),Standard Deviation (%),30-Day Average SOFR,90-Day Average SOFR,180-Day Average SOFR,SOFR Index,Revision Indicator (Y/N),Footnote ID,Quarter
0,12/30/2022,SOFR,4.3,4.20,4.26,4.33,4.48,1004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Q4_2022
1,12/29/2022,SOFR,4.3,4.20,4.26,4.32,4.45,1037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,12/28/2022,SOFR,4.3,4.21,4.27,4.32,4.45,1005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,12/27/2022,SOFR,4.3,4.15,4.27,4.32,4.40,1048,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,12/23/2022,SOFR,4.3,4.21,4.26,4.31,4.38,991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [35]:
SOFR_rates = SOFR_rates[SOFR_rates['Quarter'] != '0']


In [36]:
SOFR_rates = SOFR_rates[['Effective Date','Rate (%)','Quarter']]
SOFR_rates.head()


,Effective Date,Rate (%),Quarter
0,12/30/2022,4.30,Q4_2022
61,09/30/2022,2.98,Q3_2022
125,06/30/2022,1.50,Q2_2022
187,03/31/2022,0.29,Q1_2022
249,12/31/2021,0.05,Q4_2021


In [37]:
SOFR_rates.to_csv(r"/Users/erintoomey/PycharmProjects/pythonProject/Datasource/SOFR_rates_cleaned.csv", index=False)

My goal for this project was to analyze certain financial statement accounts in different industries over time and see if any patterns came up with changing market rates. After cleaning and joining all of my data, I imported my final CSV to Tableau and found a lot of issues with the data. I discovered that the taxonomy used to tag accounts changed about halfway through 2022, so many of the accounts I wanted to look into actually were tagged with a different name in Q1 and Q2 than they were in Q3 and Q4, so I couldn't get a complete picture of any one account throughout the year. Additionally, since filings commonly include balances for the current quarter and prior year end, the Q4 2021 balances are basically being picked up 4 times, so that limits the time period I wanted to look at by a quarter. Unfortunately I decided this data was not fit for analysis in the way that I had hoped.